In [1]:
#the below two lines of code are only for display purposes when using jupyter notebook. If using another IDE, you can delete the next two lines
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
col_names = ['participant_age', 'participant_gender', 'participant_type', 'participant_status']

# # for many rows, the number of ages given is less than the number of characteristics given. 
# # in this case, throw out the row. this is to avoid the following:
# # 1. assigning an exact age to some characteristic arbitarily
# # 2. forgoing using exact ages altogether.
# for i in range(len(col_names)): 
#     if (i > 0):
#         df = df[df['participant_age'].map(len)-df[col_names[i]].map(len)==0]
# for col in col_names:
#     df = df[df[col]!='NO_DATA']
    
# df = df.reset_index(drop=True)

# # create a new table that has one row per person/participant (instead of one row per incident)
# # capture incident ID so this can be merged with main dataset as needed

# # create "seed table" from first row
# ages     = list(df.participant_age[0])
# genders  = list(df.participant_gender[0])
# types    = list(df.participant_status[0])
# statuses = list(df.participant_type[0])
# d = {'Age':ages, 'Gender':genders, 'Type':types, 'Status':statuses}
# person_tbl = pd.DataFrame(d)
# person_tbl['incident_id'] = df.incident_id[0]
# c = 0
# for i in range(len(df)):
#     if (i > 0):
#         ages     = list(df.participant_age[i])
#         genders  = list(df.participant_gender[i])
#         types    = list(df.participant_status[i])
#         statuses = list(df.participant_type[i])
#         d = {'Age':ages, 'Gender':genders, 'Type':types, 'Status':statuses}
#         tbl = pd.DataFrame(d)
#         tbl['incident_id'] = df.incident_id[i]
#         person_tbl = pd.concat([person_tbl, tbl])
#         c = c + 1
#         p = c/len(df)*100
#         print(str(c) + " rows processed, " + str(round(p, 1)) + "% complete", end='\r')
# # write to disk so the above slow-running for loop only has to be done once
# person_tbl.to_csv('person_tbl.csv', index=False)

In [1]:
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
import altair as alt
from altair import datum
import site
import sys
site.addsitedir(r'C:\Users\pcallahan\Desktop\school\580\gun-violence-data-master\p_callahan')
import load
alt.data_transformers.enable('json')

df = load.Main_Dataset()

person_tbl = pd.read_csv('person_tbl.csv', dtype=object)
for col in person_tbl.columns:
    person_tbl = person_tbl[~person_tbl[col].str.contains("NO_DATA")]

nonsense_types = ['Injured, Unharmed', 'Killed, Injured', 'Killed, Unharmed']
for item in nonsense_types:
    person_tbl = person_tbl[~person_tbl['Status'].str.contains(item)]

person_tbl['Status'] = person_tbl['Status'].str.replace(', Arrested', '')

person_tbl = person_tbl[~person_tbl['Gender'].str.contains('Male, female')]

for col in person_tbl.columns:
    if (col == 'Gender') or (col == 'Type') or (col == 'Status'):
        person_tbl = load.pluck_digits(person_tbl, col)

person_tbl['Age'] = person_tbl['Age'].astype(float)
person_tbl = person_tbl[(person_tbl['Age']>=1) & (person_tbl['Age']<=117) | (person_tbl['Age'] == 0)]
person_tbl['Age'] = person_tbl['Age'].astype(int)
person_tbl = pd.merge(person_tbl, df, how = 'left', on='incident_id')

to_drop = ['participant_name','participant_age','participant_age_group','participant_gender','participant_type','participant_status']

for col in person_tbl:
    if col in to_drop:
        person_tbl = person_tbl.drop(col, 1)

person_tbl

Filtering process eliminated 0.0% of the initial data 



,Age,Gender,Type,Status,incident_id,date,state,city_or_county,address,n_killed,...,sources,state_house_district,state_senate_district,incident_characteristics,prim_ichars,sec_ichars,state_abr,state_pop,population,vega_state_id
0,31,Male,Subject-Suspect,Unharmed,478855,1/1/2013,Ohio,Lorain,1776 East 28th Street,1,...,http://www.morningjournal.com/general-news/201...,56,13,"[Shot - Wounded/Injured, Shot - Dead (murder, ...","[Shot - Wounded/Injured, Shot - Dead (murder, ...",[Bar/club incident - in or around establishment],OH,762062,762062,38
1,33,Male,Victim,Killed,478855,1/1/2013,Ohio,Lorain,1776 East 28th Street,1,...,http://www.morningjournal.com/general-news/201...,56,13,"[Shot - Wounded/Injured, Shot - Dead (murder, ...","[Shot - Wounded/Injured, Shot - Dead (murder, ...",[Bar/club incident - in or around establishment],OH,762062,762062,38
2,34,Male,Victim,Injured,478855,1/1/2013,Ohio,Lorain,1776 East 28th Street,1,...,http://www.morningjournal.com/general-news/201...,56,13,"[Shot - Wounded/Injured, Shot - Dead (murder, ...","[Shot - Wounded/Injured, Shot - Dead (murder, ...",[Bar/club incident - in or around establishment],OH,762062,762062,38
3,33,Male,Victim,Injured,478855,1/1/2013,Ohio,Lorain,1776 East 28th Street,1,...,http://www.morningjournal.com/general-news/201...,56,13,"[Shot - Wounded/Injured, Shot - Dead (murder, ...","[Shot - Wounded/Injured, Shot - Dead (murder, ...",[Bar/club incident - in or around establishment],OH,762062,762062,38
4,29,Female,Victim,Killed,478925,1/5/2013,Colorado,Aurora,16000 block of East Ithaca Place,4,...,http://denver.cbslocal.com/2013/01/06/officer-...,40,28,"[Shot - Dead (murder, accidental, suicide), Of...","[Shot - Dead (murder, accidental, suicide)]","[Officer Involved Incident, Officer Involved S...",CO,5758736,5758736,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161405,25,Female,Subject-Suspect,Unharmed,1083142,3/31/2018,Louisiana,Rayne,North Riceland Road and Highway 90,0,...,http://www.klfy.com/news/local/rayne-woman-cha...,NO_DATA,NO_DATA,[Shots Fired - No Injuries],[Shots Fired - No Injuries],[],LA,4467673,4467673,21
161406,21,Male,Victim,Injured,1083151,3/31/2018,Louisiana,Gretna,1300 block of Cook Street,0,...,http://www.nola.com/crime/index.ssf/2018/04/sh...,85,7,[Shot - Wounded/Injured],[Shot - Wounded/Injured],[],LA,4467673,4467673,21
161407,42,Male,Victim,Killed,1082514,3/31/2018,Texas,Houston,12630 Ashford Point Dr,1,...,http://www.khou.com/article/news/hpd-investiga...,149,17,"[Shot - Dead (murder, accidental, suicide)]","[Shot - Dead (murder, accidental, suicide)]",[],TX,6829174,6829174,47
161408,58,Female,Victim,Killed,1081940,3/31/2018,Maine,Norridgewock,434 Skowhegan Rd,2,...,https://www.centralmaine.com/2018/03/31/police...,111,3,"[Shot - Dead (murder, accidental, suicide), Su...","[Shot - Dead (murder, accidental, suicide)]","[Suicide^, Murder/Suicide, Domestic Violence]",ME,4648794,4648794,22


In [2]:
alt.Chart(person_tbl).mark_boxplot().encode(
    x = 'Type:N',
    y = 'Age:Q'
).properties(
    height = 250,
    width = 500,
    title = 'Age Distributions for Participant Types in Gun Violence Incidents'
)

alt.Chart(...)

In [15]:
status_bd = person_tbl['Status'].str.split('|', expand=True).stack().value_counts().reset_index()
status_bd

,index,0
0,Unharmed,70377
1,Injured,44581
2,Killed,40474
3,Arrested,5978


In [5]:
person_tbl['Age'] = np.dtype('int64').type(person_tbl['Age'])
from bioinfokit.analys import stat
res = stat()

res.anova_stat(df=person_tbl, res_var='Age', anova_model='Age ~ C(Status)')
res.anova_summary
# output


,sum_sq,df,F,PR(>F)
C(Status),8.621804e+05,3.0,1708.188454,0.0
Residual,2.715569e+07,161406.0,NaN,NaN


In [3]:
import statsmodels.stats.multicomp as multi

mcDate = multi.MultiComparison(person_tbl['Age'],person_tbl['Status'])
Results = mcDate.tukeyhsd()
print(Results)

  Multiple Comparison of Means - Tukey HSD, FWER=0.05  
 group1   group2  meandiff p-adj  lower   upper  reject
-------------------------------------------------------
Arrested  Injured  -1.4018 0.001 -1.8607 -0.9428   True
Arrested   Killed   4.6801 0.001  4.2184  5.1419   True
Arrested Unharmed   0.1143   0.9 -0.3346  0.5632  False
 Injured   Killed   6.0819 0.001  5.8531  6.3107   True
 Injured Unharmed   1.5161 0.001  1.3143  1.7178   True
  Killed Unharmed  -4.5659 0.001 -4.7737  -4.358   True
-------------------------------------------------------


In [4]:
alt.Chart(person_tbl).mark_boxplot().encode(
    x = 'Status:N',
    y = 'Age:Q'
).properties(
    height = 250,
    width = 400,
    title = 'Age Distributions for Participant Statuses in Gun Violence Incidents'
)

alt.Chart(...)

In [5]:
alt.Chart(person_tbl).mark_boxplot().encode(
    x = 'Gender:N',
    y = 'Age:Q'
).properties(
    height = 250,
    width = 200,
    title = 'Age Distributions by Participant Gender in Gun Violence Incidents'
)

alt.Chart(...)

In [14]:
ages = person_tbl['Age'].to_list()
hf.normaltest_Print(ages, 'Ages of Gun Crime Participants')


Ages of Gun Crime Participants--- Dataset Stats:
Size of Ages of Gun Crime Participants Dataset = 161410
Normality Test-->

p value = 0.0

We reject the null hypothesis that the distribution of values in the 'Ages of Gun Crime Participants' dataset is normal.


In [10]:
alt.Chart(person_tbl).mark_bar().encode(
    alt.X("Age:Q",
    bin=alt.Bin(extent=[0, 120], step=10),
    scale = alt.Scale(domain=(0,100))
    ),
    y = 'count():Q'
)

alt.Chart(...)

In [11]:
alt.Chart(person_tbl).mark_bar().encode(
    x = alt.X('state:N', sort='-y'),
    y = 'count():Q',
    color = 'Type:N'
).properties(
    title = 'Distribution of Female Victims and Suspects in Gun Crime, by State'
).transform_filter(
    (datum['Gender'] == 'Female')
)

alt.Chart(...)

In [12]:
alt.Chart(person_tbl).mark_bar().encode(
    x = alt.X('state:N', sort='-y'),
    y = 'count():Q',
    color = 'Type:N'
).properties(
    title = 'Distribution of Male Victims and Suspects in Gun Crime, by State'
).transform_filter(
    (datum['Gender'] == 'Male')
)

alt.Chart(...)

In [15]:
ave_age_by_state = person_tbl.groupby('state')['Age'].mean()
ave_age_by_state = ave_age_by_state.reset_index()
ave_age_by_state.dtypes
ave_age_by_state.columns = ['AveAge' if x == 'Age' else x for x in ave_age_by_state.columns]
ave_age_by_state = ave_age_by_state.sort_values(by='AveAge', ascending = False)
ave_age_by_state.reset_index(inplace = True, drop=True)
ave_age_by_state

,state,AveAge
0,Montana,37.680751
1,Wyoming,36.943005
2,Hawaii,35.895105
3,Vermont,35.489231
4,Maine,34.891213
5,Idaho,34.239051
6,West Virginia,33.720299
7,Nevada,33.591154
8,Oregon,32.933482
9,Arizona,32.900804


In [16]:
state_sort = ave_age_by_state.state.to_list()
alt.Chart(person_tbl).mark_bar().encode(
    x = 'Type:N',
    y = 'mean(Age):Q',
    column = 'state:N',
    tooltip = ['mean(Age):Q','Type:N']
).properties(
    title = 'Distribution of Ages for Gun Crime Suspects, by State'
)

alt.Chart(...)

In [17]:
state_sort = ave_age_by_state.state.to_list()
alt.Chart(person_tbl).mark_boxplot().encode(
    x = alt.X('state:N'),
    y = 'Age:Q'
).properties(
    title = 'Distribution of Ages for Gun Crime Victim, by State'
).transform_filter(
    (datum['Type'] == 'Victim')
)

alt.Chart(...)